In [20]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from utilities import set_multiple_columns_datatype
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
from sklearn.svm import SVC

In [21]:
#Import data
test = pd.read_csv('data/test.csv')
train = pd.read_csv('data/train.csv')

In [22]:
columns = {"Pclass":'category', 'Embarked':'category', "Sex":'category'}
train = set_multiple_columns_datatype(train, columns)

In [23]:
#Inspect
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   PassengerId  891 non-null    int64   
 1   Survived     891 non-null    int64   
 2   Pclass       891 non-null    category
 3   Name         891 non-null    object  
 4   Sex          891 non-null    category
 5   Age          714 non-null    float64 
 6   SibSp        891 non-null    int64   
 7   Parch        891 non-null    int64   
 8   Ticket       891 non-null    object  
 9   Fare         891 non-null    float64 
 10  Cabin        204 non-null    object  
 11  Embarked     889 non-null    category
dtypes: category(3), float64(2), int64(4), object(3)
memory usage: 65.8+ KB


In [24]:
train.describe()

,PassengerId,Survived,Age,SibSp,Parch,Fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,80.000000,8.000000,6.000000,512.329200


In [25]:
train_dummies = pd.get_dummies(train.drop(['Cabin', 'Name', 'Ticket'], axis=1))

In [26]:
X = train_dummies.drop(['Survived', 'PassengerId'], axis=1)
y = train['Survived']

X_train_dummies, X_test_dummies, y_train_dummies, y_test_dummies = train_test_split(X, y, test_size=0.2, random_state=42)

In [36]:

models = {
    'Logistic Regression': LogisticRegression(),
    'Random Forest': RandomForestClassifier(max_depth=6, min_samples_split=5, n_estimators=335, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
    'SVM': SVC(),
    'Naive Bayes (Gaussian)': GaussianNB(),
    'Naive Bayes (Bernoulli)': BernoulliNB(),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'XGBoost': xgb.XGBClassifier(),
    'Stacking': StackingClassifier(estimators=[('rf', RandomForestClassifier()), ('gb', GradientBoostingClassifier())], final_estimator=LogisticRegression()),
}

pipelines = {}
for name, model in models.items():
    pipelines[name] = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler()),
        ('classifier', model)
    ])

In [37]:
results = {}
for name, pipeline in pipelines.items():
    scores = cross_val_score(pipeline, X_train_dummies, y_train_dummies, cv=5, scoring='accuracy')
    results[name] = scores
    print(f'{name}: {scores.mean():.2f} ± {scores.std():.2f}')


Logistic Regression: 0.79 ± 0.02
Random Forest: 0.83 ± 0.02
Gradient Boosting: 0.82 ± 0.02
SVM: 0.82 ± 0.03
Naive Bayes (Gaussian): 0.78 ± 0.03
Naive Bayes (Bernoulli): 0.78 ± 0.03
K-Nearest Neighbors: 0.80 ± 0.03
XGBoost: 0.80 ± 0.02
Stacking: 0.82 ± 0.01


In [ ]:
for name, pipeline in pipelines.items():
    pipeline.fit(X_train_dummies, y_train_dummies)
    y_pred = pipeline.predict(X_test_dummies)
    accuracy = accuracy_score(y_test_dummies, y_pred)
    precision = precision_score(y_test_dummies, y_pred)
    recall = recall_score(y_test_dummies, y_pred)
    f1 = f1_score(y_test_dummies, y_pred)
    print(f'\n{name} Performance on Test Set:')
    print(f'Accuracy: {accuracy:.2f}')
    print(f'Precision: {precision:.2f}')
    print(f'Recall: {recall:.2f}')
    print(f'F1 Score: {f1:.2f}')


In [ ]:
test_new = set_multiple_columns_datatype(test, columns)
X_true_test = pd.get_dummies(test_new.drop(['Cabin', 'Name', 'Ticket', 'PassengerId'], axis=1))
y_true_pred = pipelines['Random Forest'].predict(X_true_test)
y_true_pred

In [ ]:
test['Survived'] = y_true_pred
test


In [ ]:
results = pd.DataFrame()
results['Survived'] = test['Survived']
results['PassengerId'] = test['PassengerId']
results.to_csv('data/results.csv', index=False)

In [ ]:
sol = pd.read_csv('data/results.csv')
sol['PassengerId'].unique()